We use a try except and finally blocks to run code that can run into exceptions. The try section is where the code is executed, except to catch exceptions and finally is run at the end regardless.

We use the finally block to make sure a piece of code is executed, whether an exception has happened or not:

In [1]:
try:
    10 / 2
except ZeroDivisionError:
    print('Zero division exception occurred')
finally:
    print('finally ran!')

finally ran!


In [2]:
try:
    1 / 0
except ZeroDivisionError:
    print('Zero division exception occurred')
finally:
    print('finally ran!')

Zero division exception occurred
finally ran!


This is true even if the try catch and finally blocks are inside a function body. This is very useful when it comes to releasing resources especially when it comes to opening and closing of files.

In [3]:
#there is no such file
try:
    f = open('test.txt', 'w')
    a = 1 / 0
except:
    print('an exception occurred...')
finally:
    print('Closing file...')
    f.close()


an exception occurred...
Closing file...


In [4]:
#this file exists
try:
    f = open('notebook2script.py', 'w')
    a = 1 / 0
except:
    print('an exception occurred...')
finally:
    print('Closing file...')
    f.close()


an exception occurred...
Closing file...


This is just a lot of code for a simple task and for repetitive tasks is not a good solution as it can lead to a bunch of mistakes.
Context managers
Run some code to create some object(s)
Work with object(s)
Run some code when done to clean up object(s)

They basically perform the same try except finally pattern under the hood and provide a clean implementation to the end user

In [5]:
with open('test.txt', 'w') as file:
    print('inside with: file closed?', file.closed)
print('after with: file closed?', file.closed) # this is not in the with scope hence file is closed.

inside with: file closed? False
after with: file closed? True


Context managers can be used for more than just opening and closing files.

If we think about it there are two phases to a context manager:

when the with statement is executing: we enter the context
when the with block is done: we exit the context
We can create our own context manager using a class that implements an __enter__ method which is executed when we enter the context, and an __exit__ method that is executed when we exit the context.

There is a general pattern that context managers can help us deal with:

Open - Close
Lock - Release
Change - Reset
Enter - Exit
Start - Stop
The __enter__ method is quite straightforward. It can (but does not have to) return one or more objects we then use inside the with block.

The __exit__ method however is slightly more complicated.

It needs to return a boolean True/False. This indicates to Python whether to suppress any errors that occurred in the with block. As we saw with files, that was not the case - i.e. it returns a False
If an error does occur in the with block, the error information is passed to the __exit__ method - so it needs three things: the exception type, the exception value and the traceback. If no error occured, then those values will simply be None.


In [6]:
class MyContext:
    def __init__(self):
        self.obj = None
        
    def __enter__(self):
        print('entering context...')
        self.obj = 'the Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting context...')
        if exc_type:
            print(f'*** Error occurred: {exc_type}, {exc_value}')
        return True  # suppress exceptions

In [7]:
with MyContext() as obj:
    raise ValueError
print('reached here without an exception...')

entering context...
exiting context...
*** Error occurred: <class 'ValueError'>, 
reached here without an exception...


In [8]:
class MyContext:
    def __init__(self):
        self.obj = None
        
    def __enter__(self):
        print('entering context...')
        self.obj = 'the Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting context...')
        if exc_type:
            print(f'*** Error occurred: {exc_type}, {exc_value}')
        return False  # do not suppress exceptions

In [10]:
with MyContext() as obj:
    raise ValueError('I raised a Value error')

entering context...
exiting context...
*** Error occurred: <class 'ValueError'>, I raised a Value error


ValueError: I raised a Value error

Note that the __enter__ method can return anything, including the context manager itself.

If we wanted to, we could re-write our file context manager this way:

In [11]:
??file

In [13]:
class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opening file...')
        self.file = open(self.name, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('closing file...')
        self.file.close()
        return False

In [14]:
with File('test.txt', 'w') as f:
    f.write('This is a late parrot!')

opening file...
closing file...


In [15]:

class File():
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opening file...')
        self.file = open(self.name, self.mode)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('closing file...')
        self.file.close()
        return False

In [18]:
with File('test.txt', 'r') as file_ctx:
    print(next(file_ctx.file))
    print(file_ctx.name)
    print(file_ctx.mode)

opening file...
This is a late parrot!
test.txt
r
closing file...


In [19]:
??file